### 1 BANK RENEGE

In [1]:
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'CLIENTE %02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Aqui estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Esperado %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Terminado' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: Renegado despues %6.3f' % (env.now, name, wait))


# Setup and start the simulation
print('RENEGO BANCARIO')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

RENEGO BANCARIO
 0.0000 CLIENTE 00: Aqui estoy
 0.0000 CLIENTE 00: Esperado  0.000
 3.8595 CLIENTE 00: Terminado
10.2006 CLIENTE 01: Aqui estoy
10.2006 CLIENTE 01: Esperado  0.000
12.7265 CLIENTE 02: Aqui estoy
13.9003 CLIENTE 02: Renegado despues  1.174
23.7507 CLIENTE 01: Terminado
34.9993 CLIENTE 03: Aqui estoy
34.9993 CLIENTE 03: Esperado  0.000
37.9599 CLIENTE 03: Terminado
40.4798 CLIENTE 04: Aqui estoy
40.4798 CLIENTE 04: Esperado  0.000
43.1401 CLIENTE 04: Terminado


### 2 CARWASH



In [2]:
import random

import simpy


RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes


class Carwash(object):
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME)
        print("El lavadero de carros elimino el %d%% de la suciedad del %s" %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    print('%s llega al lavadero de carros en %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavadero de carros en %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s deja el lavadero de carros en %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Create a carwash, a number of initial cars and keep creating cars
    approx. every ``t_inter`` minutes."""
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Setup and start the simulation
print('LAVADERO DE CARROS')
#print('Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

LAVADERO DE CARROS
Carro 0 llega al lavadero de carros en 0.00.
Carro 1 llega al lavadero de carros en 0.00.
Carro 2 llega al lavadero de carros en 0.00.
Carro 3 llega al lavadero de carros en 0.00.
Carro 0 entra al lavadero de carros en 0.00.
Carro 1 entra al lavadero de carros en 0.00.
Carro 4 llega al lavadero de carros en 5.00.
El lavadero de carros elimino el 97% de la suciedad del Carro 0
El lavadero de carros elimino el 67% de la suciedad del Carro 1
Carro 0 deja el lavadero de carros en 5.00.
Carro 1 deja el lavadero de carros en 5.00.
Carro 2 entra al lavadero de carros en 5.00.
Carro 3 entra al lavadero de carros en 5.00.
Carro 5 llega al lavadero de carros en 10.00.
El lavadero de carros elimino el 64% de la suciedad del Carro 2
El lavadero de carros elimino el 58% de la suciedad del Carro 3
Carro 2 deja el lavadero de carros en 10.00.
Carro 3 deja el lavadero de carros en 10.00.
Carro 4 entra al lavadero de carros en 10.00.
Carro 5 entra al lavadero de carros en 10.00.
El l

### 3 MACHINE SHOP

In [3]:
import random

import simpy


RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Return actual processing time for a concrete part."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Return time until next failure for a machine."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *repairman* and continues the production
    after the it is repaired.

    A machine has a *name* and a numberof *parts_made* thus far.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce parts as long as the simulation runs.

        While making a part, the machine may break multiple times.
        Request a repairman when this happens.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Break the machine every now and then."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """The repairman's other (unimportant) job."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Setup and start the simulation
print('TIENDA DE MAQUINA')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Maquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Resultados de la tienda de maquinas despues de %s semanas' % WEEKS)
for machine in machines:
    print('%s hizo %d partes.' % (machine.name, machine.parts_made))

TIENDA DE MAQUINA
Resultados de la tienda de maquinas despues de 4 semanas
Maquina 0 hizo 3251 partes.
Maquina 1 hizo 3273 partes.
Maquina 2 hizo 3242 partes.
Maquina 3 hizo 3343 partes.
Maquina 4 hizo 3387 partes.
Maquina 5 hizo 3244 partes.
Maquina 6 hizo 3269 partes.
Maquina 7 hizo 3185 partes.
Maquina 8 hizo 3302 partes.
Maquina 9 hizo 3279 partes.


### 4 MOVIE RENEGE

In [4]:
import collections
import random

import simpy


RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SIM_TIME = 120  # Simulate until


def moviegoer(env, movie, num_tickets, theater):
    """A moviegoer tries to by a number of tickets (*num_tickets*) for
    a certain *movie* in a *theater*.

    If the movie becomes sold out, she leaves the theater. If she gets
    to the counter, she tries to buy a number of tickets. If not enough
    tickets are left, she argues with the teller and leaves.

    If at most one ticket is left after the moviegoer bought her
    tickets, the *sold out* event for this movie is triggered causing
    all remaining moviegoers to leave.

    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn of if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(0.5)
            env.exit()

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Setup and start the simulation
print('RENEGAR PELICULA')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Python sin cobertura', 'Proceso de matanza', 'Implementacion de la pulpa']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('La pelicula "%s" se vendio %.1f minutos después de la apertura del mostrador de boletos. '
                 % (movie, theater.when_sold_out[movie]))
        print('  Número de personas que abandonan la cola cuando la película se agotó: %s' %
              theater.num_renegers[movie])

RENEGAR PELICULA
La pelicula "Python sin cobertura" se vendio 38.0 minutos después de la apertura del mostrador de boletos. 
  Número de personas que abandonan la cola cuando la película se agotó: 16
La pelicula "Proceso de matanza" se vendio 43.0 minutos después de la apertura del mostrador de boletos. 
  Número de personas que abandonan la cola cuando la película se agotó: 5
La pelicula "Implementacion de la pulpa" se vendio 28.0 minutos después de la apertura del mostrador de boletos. 
  Número de personas que abandonan la cola cuando la película se agotó: 5


### 5 GAS STATION REFUELING

In [5]:
import itertools
import random

import simpy


RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds


def car(name, env, gas_station, fuel_pump):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of gas from it. If the stations reservoir is
    depleted, the car has to wait for the tank truck to arrive.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la estacion de servicio en %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Request one of the gas pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s termino de respostar %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Periodically check the level of the *fuel_pump* and call the tank
    truck if the level falls below a threshold."""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print('Llamar camión cisterna a %d' % env.now)
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds


def tank_truck(env, fuel_pump):
    """Arrives at the gas station after a certain delay and refuels it."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Camión cisterna llegando al tiempo  %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Camión cisterna de reabastecimiento de combustible %.1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))


# Setup and start the simulation
print('REABASTECIMIENTO DE GASOLINERIA')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

REABASTECIMIENTO DE GASOLINERIA
Carro 0 llegando a la estacion de servicio en 87.0
Carro 0 termino de respostar 18.5 segundos.
Carro 1 llegando a la estacion de servicio en 129.0
Carro 1 termino de respostar 19.0 segundos.
Carro 2 llegando a la estacion de servicio en 284.0
Carro 2 termino de respostar 21.0 segundos.
Carro 3 llegando a la estacion de servicio en 385.0
Carro 3 termino de respostar 13.5 segundos.
Carro 4 llegando a la estacion de servicio en 459.0
Llamar camión cisterna a 460
Carro 4 termino de respostar 22.0 segundos.
Carro 5 llegando a la estacion de servicio en 705.0
Carro 6 llegando a la estacion de servicio en 750.0
Camión cisterna llegando al tiempo  760
Camión cisterna de reabastecimiento de combustible 188.0 litros.
Carro 6 termino de respostar 29.0 segundos.
Carro 5 termino de respostar 76.5 segundos.
Carro 7 llegando a la estacion de servicio en 891.0
Carro 7 termino de respostar 13.0 segundos.


### 6 PROCESS COMMUNICATION

In [6]:
import random

import simpy


RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """A Broadcast pipe that allows one process to send messages to many.

    This construct is useful when message consumers are running at
    different rates than message generators and provides an event
    buffering to the consuming processes.

    The parameters are used to create a new
    :class:`~simpy.resources.store.Store` instance each time
    :meth:`get_output_conn()` is called.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Get a new output connection for this broadcast pipe.

        The return value is a :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s dice hola a las %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('Tardar en recibir un mensaje: en el tiempo %d: %s mensaje recibido: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('En el timpo %d: %s Mensaje recibido: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('PROCESO DE COMUNICACION')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generador A', env, pipe))
env.process(message_consumer('Consumidor A', env, pipe))

print('\nComunicación de tubería uno a uno\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generador A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nComunicación de tubería de uno a muchos\n')
env.run(until=SIM_TIME)

PROCESO DE COMUNICACION

Comunicación de tubería uno a uno

En el timpo 6: Consumidor A Mensaje recibido: Generador A dice hola a las 6.
En el timpo 12: Consumidor A Mensaje recibido: Generador A dice hola a las 12.
En el timpo 19: Consumidor A Mensaje recibido: Generador A dice hola a las 19.
En el timpo 26: Consumidor A Mensaje recibido: Generador A dice hola a las 26.
En el timpo 36: Consumidor A Mensaje recibido: Generador A dice hola a las 36.
En el timpo 46: Consumidor A Mensaje recibido: Generador A dice hola a las 46.
En el timpo 52: Consumidor A Mensaje recibido: Generador A dice hola a las 52.
En el timpo 58: Consumidor A Mensaje recibido: Generador A dice hola a las 58.
Tardar en recibir un mensaje: en el tiempo 66: Consumidor A mensaje recibido: Generador A dice hola a las 65
En el timpo 75: Consumidor A Mensaje recibido: Generador A dice hola a las 75.
En el timpo 85: Consumidor A Mensaje recibido: Generador A dice hola a las 85.
En el timpo 95: Consumidor A Mensaje recibi

### 7 EVENT LATENCY

In [7]:
import simpy


SIM_DURATION = 100


class Cable(object):
    """This class represents the propagation through a cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('el remitente lo envio a los %d' % env.now)


def receiver(env, cable):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Recibio esto a los %d mientras que %s' % (env.now, msg))


# Setup and start the simulation
print('LATENCIA DEL EVENTO')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

LATENCIA DEL EVENTO
Recibio esto a los 15 mientras que el remitente lo envio a los 5
Recibio esto a los 20 mientras que el remitente lo envio a los 10
Recibio esto a los 25 mientras que el remitente lo envio a los 15
Recibio esto a los 30 mientras que el remitente lo envio a los 20
Recibio esto a los 35 mientras que el remitente lo envio a los 25
Recibio esto a los 40 mientras que el remitente lo envio a los 30
Recibio esto a los 45 mientras que el remitente lo envio a los 35
Recibio esto a los 50 mientras que el remitente lo envio a los 40
Recibio esto a los 55 mientras que el remitente lo envio a los 45
Recibio esto a los 60 mientras que el remitente lo envio a los 50
Recibio esto a los 65 mientras que el remitente lo envio a los 55
Recibio esto a los 70 mientras que el remitente lo envio a los 60
Recibio esto a los 75 mientras que el remitente lo envio a los 65
Recibio esto a los 80 mientras que el remitente lo envio a los 70
Recibio esto a los 85 mientras que el remitente lo envio 